In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CSV file
data = pd.read_csv('_annotations11.csv')   # dataset
data_test = pd.read_csv('_annotatetest11.csv') #testset

# Split the data into images and labels
filenames = data['filename']        # dataset
labels = data['class']
xmins = data['xmin']
ymins = data['ymin']
xmaxs = data['xmax']
ymaxs = data['ymax']

filenames_test = data_test['filename']         #testset
labels_test = data_test['class']       
xmins_test = data_test['xmin']
ymins_test = data_test['ymin']
xmaxs_test = data_test['xmax']
ymaxs_test = data_test['ymax']

# Convert labels to integers
label_to_int = {label: i for i, label in enumerate(labels.unique())}
int_to_label = {i: label for label, i in label_to_int.items()}
labels = labels.map(label_to_int)

# Preprocess the images for dataset
images = []
segmented_images = []
segmented_labels = []

for i in range(len(filenames)):
    filename = filenames[i]
    xmin = xmins[i]
    ymin = ymins[i]
    xmax = xmaxs[i]
    ymax = ymaxs[i]
    
    image = load_img(filename, target_size=(32, 32))
    image = img_to_array(image) / 255.0  # Normalize pixel values to [0, 1]
    images.append(image)
    
    # Perform character segmentation
    plate_image = load_img(filename)
    character_image = plate_image.crop((xmin, ymin, xmax, ymax))
    character_image = character_image.resize((32, 32))
    character_array = img_to_array(character_image) / 255.0
    segmented_images.append(character_array)
    segmented_labels.append(labels[i])
    

images = np.array(images)
segmented_images = np.array(segmented_images)
segmented_labels = np.array(segmented_labels)

# Preprocess the images for testset
images_test = []
segmented_images_test = []
segmented_labels_test = []

for i in range(len(filenames_test)):
    filename_test = filenames_test[i]
    xmin_test = xmins_test[i]
    ymin_test = ymins_test[i]
    xmax_test = xmaxs_test[i]
    ymax_test = ymaxs_test[i]
    
    image_test = load_img(filename_test, target_size=(32, 32))
    image_test = img_to_array(image_test) / 255.0  # Normalize pixel values to [0, 1]
    images_test.append(image_test)
    
    # Perform character segmentation
    plate_image_test = load_img(filename_test)
    character_image_test = plate_image_test.crop((xmin_test, ymin_test, xmax_test, ymax_test))
    character_image_test = character_image_test.resize((32, 32))
    character_array_test = img_to_array(character_image_test) / 255.0
    segmented_images_test.append(character_array_test)
    segmented_labels_test.append(labels_test[i])

images_test = np.array(images_test)
segmented_images_test = np.array(segmented_images_test)
segmented_labels_test = np.array(segmented_labels_test)

# Split the data into training and testing sets
#train_images, test_images, train_labels, test_labels = train_test_split(segmented_images, segmented_labels, test_size=0.2, random_state=42)
train_images=segmented_images
train_labels=segmented_labels

test_images=segmented_images_test
test_labels=segmented_labels_test

# Apply data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)
datagen.fit(train_images)

# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(label_to_int), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model with data augmentation
model.fit(datagen.flow(train_images, train_labels, batch_size=32),
          epochs=20)
          #validation_data=(test_images, test_labels))

'''
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('Test accuracy:', test_acc)
'''

# Make predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Map the predicted class indices back to the corresponding labels
predicted_labels = [int_to_label[idx] for idx in predicted_classes]

# Combine the filenames and their corresponding predicted labels
results = [f"Filename: {filenames_test[i]}, Predicted label: {predicted_labels[i]}, Actual label: {labels_test[i]}" for i in range(len(test_images))]  

# Join the results into a single string
output_string = "\n".join(results)
print(output_string)

